# Assignment 3 - Text Analysis
An explanation this assignment could be found in the .pdf explanation document


## Materials to review for this assignment
<h4>From Moodle:</h4> 
<h5><u>Review the notebooks regarding the following python topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Working with strings</b> (tutorial notebook)<br/>
&#x2714; <b>Text Analysis</b> (tutorial notebook)<br/>
&#x2714; <b>Hebrew text analysis tools (tokenizer, wordnet)</b> (moodle example)<br/>
&#x2714; <b>(brief review) All previous notebooks</b><br/>
</div> 
<h5><u>Review the presentations regarding the following topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Text Analysis</b> (lecture presentation)<br/>
&#x2714; <b>(brief review) All other presentations</b><br/>
</div>

## Personal Details:

In [108]:
# Details Student 1:
# Name: Peleg Levy
# ID: 315877639



## Preceding Step - import modules (packages)
This step is necessary in order to use external modules (packages). <br/>

In [291]:
# --------------------------------------
import pandas as pd
import numpy as np
# --------------------------------------


# --------------------------------------
# ------------- visualizations:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
# --------------------------------------


# ---------------------------------------
import sklearn
from sklearn import preprocessing, metrics, pipeline, model_selection, feature_extraction 
from sklearn import naive_bayes, linear_model, svm, neural_network, neighbors, tree
from sklearn import decomposition, cluster

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score, silhouette_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder

from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron, SGDClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
# ---------------------------------------


# ----------------- output and visualizations: 
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)
# show several prints in one cell. This will allow us to condence every trick in one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# ---------------------------------------

### Text analysis and String manipulation imports:

In [292]:
# --------------------------------------
# --------- Text analysis and Hebrew text analysis imports:
# vectorizers:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# regular expressions:
import re
# --------------------------------------

### (optional) Hebrew text analysis - WordNet (for Hebrew)
Note: the WordNet is not a must

#### (optional) Only if you didn't install Wordnet (for Hebrew) use:

In [293]:
# word net installation:

# unmark if you want to use and need to install
!pip install wn
!python -m wn download omw-he:1.4


Cached file found: C:\Users\Peleg levy\.wn_data\downloads\7ecf10e89326bc0ac26ad94b40fe60a7b6ac3302

Checking C:\Users\PELEGL~1\AppData\Local\Temp\tmpa9meybco\omw-he\omw-he.xml
Skipping omw-he:1.4 (Hebrew Wordnet); already added



In [294]:
# word net import:

# unmark if you want to use:
import wn

### (optional) Hebrew text analysis - hebrew_tokenizer (Tokenizer for Hebrew)
Note: the hebrew_tokenizer is not a must

#### (optional) Only if you didn't install hebrew_tokenizer use:

In [295]:
# Hebrew tokenizer installation:

# unmark if you want to use and need to install:
!pip install hebrew_tokenizer

In [296]:
# Hebrew tokenizer import:

# unmark if you want to use:
import hebrew_tokenizer as ht

### Reading input files
Reading input files for train annotated corpus (raw text data) corpus and for the test corpus

In [366]:
train_filename = 'annotated_corpus_for_train.csv'
test_filename  = 'corpus_for_test.csv'
df_train = pd.read_csv(train_filename, index_col=None, encoding='utf-8')
df_test  = pd.read_csv(test_filename, index_col=None, encoding='utf-8')

In [367]:
df_train.head(8)
df_train.shape

,story,gender
0,"כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...",m
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,m
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,f
3,"כשהייתי ילד, מטוסים היה הדבר שהכי ריתק אותי. ב...",m
4,‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...,f
5,לפני כ3 חודשים טסתי לרומא למשך שבוע. טסתי במטו...,f
6,אני כבר שנתיים נשוי והשנה אני ואישתי סוף סוף י...,m
7,השנה התחלנו שיפוץ בדירה שלנו בתל אביב. הדירה ה...,f


(753, 2)

In [368]:
df_test.head(3)
df_test.shape

,test_example_id,story
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""..."
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...


(323, 2)

### Your implementation:
Write your code solution in the following code-cells

In [369]:
df_train.shape
df_test.shape

(753, 2)

(323, 2)

In [370]:
#Convert categorical to numeric values
df_train_numeric = df_train.copy()
df_train_numeric['gender'] = df_train_numeric['gender'].replace({'m':0, 'f':1})


In [371]:
df_train_numeric.head(10)
df_train_numeric.shape

,story,gender
0,"כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...",0
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,0
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,1
3,"כשהייתי ילד, מטוסים היה הדבר שהכי ריתק אותי. ב...",0
4,‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...,1
5,לפני כ3 חודשים טסתי לרומא למשך שבוע. טסתי במטו...,1
6,אני כבר שנתיים נשוי והשנה אני ואישתי סוף סוף י...,0
7,השנה התחלנו שיפוץ בדירה שלנו בתל אביב. הדירה ה...,1
8,עד לפני כמה חודשים אני הייתי בטוח שכל חיי אהיה...,0
9,"בן זוגי חגג יום הולדת, ורציתי להפתיע אותו ולסג...",1


(753, 2)

In [372]:
#Split the numeric training data to X and Y
X = df_train_numeric.drop('gender', axis=1)
y = df_train_numeric['gender']

In [373]:
X.shape
y.shape

(753, 1)

(753,)

In [374]:
y.head(5)

0    0
1    0
2    1
3    0
4    1
Name: gender, dtype: int64

In [375]:
#Now, we will split the numeric X and  y to test and train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, shuffle=False)

In [376]:
#Create the x train test divided by words & Some feature selection
# Extracting Features using CountVectorizer
array = X_train["story"].to_numpy()
vec = CountVectorizer(min_df=4, max_df=300) #min_df =4 -> word that occur in less than 4 documents will be ignored same words that occur more than 300 times.
story = vec.fit_transform(array)
story.toarray()

# Create DataFrame, Extract words using CountVectorizer
df1 = pd.DataFrame(story.toarray(), columns=vec.get_feature_names())
result = pd.concat([df1, X_train], axis=1).reindex(X_train.index)

# Remove unuse columns
filtered_df = result.drop(['story'], axis=1)

# Remove columns that start with digits
for i in range(0,10):
    filtered_df = filtered_df.loc[:,~filtered_df.columns.str.startswith(str(i))]

# Remove columns that start with specific letters
filtered_df = filtered_df.loc[:,~filtered_df.columns.str.startswith(str("ו"))]
filtered_df = filtered_df.loc[:,~filtered_df.columns.str.startswith(str("ה"))]
filtered_df = filtered_df.loc[:,~filtered_df.columns.str.startswith(str("ל"))]
filtered_df.head()
filtered_df.shape

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

אבא  אבי  אביב  אבן  אבק  אגב  אגם  אדום  אדומות  אדם  אדמה  אה  אהב  אהבה  \
0    0    0     0    0    0    0    1     0       0    0     0   0    0     0   
1    0    0     0    0    0    0    0     0       0    3     0   0    0     0   
2    0    0     0    0    0    0    0     0       0    0     0   0    0     0   
3    5    0     0    0    0    0    0     0       0    1     0   0    0     0   
4    0    0     0    0    0    0    0     0       0    0     0   0    0     0   

   אהבו  אהבתי  אהיה  או  אוגוסט  אודות  אוהב  אוהבים  אוהבת  אוהד  אוהדים  \
0     0      0     0   0       0      0     0       0      0     0       0   
1     0      0     0   0       0      0     1       0      0     0       0   
2     0      0     0   0       0      0     0       0      0     0       0   
3     0      0     0   0       0      0     0       0      0     0       0   
4     0      0     0   1       0      0     0       0      0     0       0   

   אוהל  אוהלים  אוויר  אווירה  אוטו  אוטובוס  אוטומטי  אוכל  אוכלים  אולי  \
0     0       0      0       0     0        0        0     0       0     0   
1     1       0      0       0     0        0        0     0       0     0   
2     0       0      0       0     0        0        0     0       0     0   
3     0       0      0       0     0        0        0     0       0     0   
4     0       0      0       0     0        0        1     0       0     0   

   אולם  אולמות  אומנה  אומנם  אומץ  אומר  אומרים  אומרת  אונים  אונליין  \
0     0       0      0      1     0     1       0      0      0        0   
1     0       0      0      0     0     2       0      0      0        0   
2     0       0      0      0     0     0       0      0      0        0   
3     0       0      0      0     0     0       0      0      0        0   
4     0       0      0      0     0     0       0      0      0        0   

   אופטימיים  אופי  אופייני  אופן  אופנוע  אופניים  אופציה  אוציא  אוקיי  אור  \
0          0     0        0     0       0        0       0      0      1    0   
1          0     0        0     0       0        0       0      0      0    0   
2          0     0        0     0       0        0       0      0      0    0   
3          0     0        0     0       0        0       0      0      0    0   
4          0     0        0     0       0        0       0      0      0    0   

   אורז  אורח  אורחים  אורך  אושר  אות  אותה  אותו  אותך  אותם  אותן  אותנו  \
0     0     0       0     0     0    0     0     1     0     0     0      0   
1     0     0       0     0     0    0     0     0     0     0     0      1   
2     0     0       0     0     0    0     1     0     0     0     0      0   
3     0     0       0     0     0    0     0     3     0     0     0      0   
4     0     0       0     0     0    0     0     0     0     1     0      0   

   אז  אזור  אזורים  אזכה  אזכור  אח  אחווה  אחוז  אחוזים  אחורה  אחות  אחותה  \
0   1     0       0     0      0   0      0     0       0      0     0      0   
1   0     0       0     0      0   0      0     0       0      0     0      0   
2   1     0       0     0      0   0      0     0       0      0     0      0   
3   2     0       0     0      0   0      0     0       0      1     0      0   
4   0     0       0     0      0   0      0     0       0      0     0      0   

   אחותי  אחזור  אחי  אחיי  אחים  אחלה  אחר  אחראי  אחראית  אחרון  אחרונה  \
0      0      0    0     0     0     1    0      0       0      0       0   
1      0      0    0     0     0     0    0      0       0      0       0   
2      0      0    0     0     0     0    1      0       0      0       0   
3      0      0    0     0     0     0    0      0       0      0       0   
4      0      1    0     0     0     0    0      0       0      0       0   

   אחרונים  אחרות  אחריה  אחריו  אחריות  אחריי  אחרים  אחרינו  אחרת  אחת  \
0        0      0      0      0       0      0      0       0     0    1   
1        0      0      0      0       0      0      0    

(677, 4188)

In [377]:
# Drop rows with NaN values
#filtered_df = filtered_df.dropna()
filtered_df.shape

(677, 4188)

In [378]:

# Classify training
df2 = filtered_df;

df2 = df2[0:76]
for col in df2.columns:
    df2[col].values[:] = 0

for index, row in df2.iterrows():
    hebrew_text = X_test['story'].iloc[index]

    tokens = ht.tokenize(hebrew_text) 
    for grp, token, token_num, (start_index, end_index) in tokens:
        if token in df2:
            df2[token].iloc[index] = df2[token].iloc[index] + 1
        
df2.head()
df2.shape


אבא  אבי  אביב  אבן  אבק  אגב  אגם  אדום  אדומות  אדם  אדמה  אה  אהב  אהבה  \
0    0    0     0    0    0    0    0     0       0    0     0   0    0     1   
1    0    0     0    0    0    0    0     0       0    0     0   0    0     0   
2    0    0     0    0    0    0    0     0       0    0     0   0    1     0   
3    0    0     0    0    0    0    0     0       0    0     0   0    0     0   
4    0    0     0    0    0    0    0     0       0    0     0   0    0     0   

   אהבו  אהבתי  אהיה  או  אוגוסט  אודות  אוהב  אוהבים  אוהבת  אוהד  אוהדים  \
0     0      0     0   5       0      0     0       0      1     0       0   
1     0      0     0   0       0      0     0       0      0     0       0   
2     0      0     0   0       0      0     0       0      0     0       0   
3     0      0     0   0       0      0     0       0      0     0       0   
4     0      0     0   1       0      0     0       0      0     0       0   

   אוהל  אוהלים  אוויר  אווירה  אוטו  אוטובוס  אוטומטי  אוכל  אוכלים  אולי  \
0     0       0      0       0     0        0        0     2       0     0   
1     0       0      0       0     0        0        0     0       0     0   
2     0       0      0       0     0        0        0     0       0     0   
3     0       0      0       0     0        0        0     0       0     0   
4     0       0      0       0     0        0        0     0       0     0   

   אולם  אולמות  אומנה  אומנם  אומץ  אומר  אומרים  אומרת  אונים  אונליין  \
0     0       0      0      0     0     0       0      0      0        0   
1     0       0      0      0     0     0       0      0      0        0   
2     0       0      0      0     0     0       0      0      0        0   
3     0       0      0      0     0     0       0      0      0        0   
4     0       0      0      0     0     0       0      0      0        0   

   אופטימיים  אופי  אופייני  אופן  אופנוע  אופניים  אופציה  אוציא  אוקיי  אור  \
0          0     0        0     0       0        0       0      0      0    0   
1          0     0        0     0       0        0       0      0      0    0   
2          0     0        0     0       0        0       0      0      0    0   
3          0     0        0     0       0        0       0      0      0    0   
4          0     0        0     0       0        0       0      0      0    0   

   אורז  אורח  אורחים  אורך  אושר  אות  אותה  אותו  אותך  אותם  אותן  אותנו  \
0     0     0       0     0     0    0     0     0     0     0     0      0   
1     0     0       0     0     0    0     0     0     0     0     0      1   
2     0     0       0     0     1    0     0     1     0     0     0      0   
3     0     0       0     0     0    0     0     2     0     0     0      0   
4     0     0       0     0     0    0     3     0     0     0     0      0   

   אז  אזור  אזורים  אזכה  אזכור  אח  אחווה  אחוז  אחוזים  אחורה  אחות  אחותה  \
0   1     0       0     0      0   0      0     0       0      0     0      0   
1   0     0       0     0      0   0      0     0       0      0     0      0   
2   0     0       0     0      0   0      0     0       0      0     0      0   
3   0     0       0     0      0   0      0     0       0      0     0      0   
4   1     0       0     0      0   0      0     0       0      0     0      0   

   אחותי  אחזור  אחי  אחיי  אחים  אחלה  אחר  אחראי  אחראית  אחרון  אחרונה  \
0      0      0    0     0     0     0    0      0       0      0       0   
1      0      0    0     0     0     0    0      0       0      0       0   
2      0      1    0     0     0     0    0      0       0      0       0   
3      0      0    0     0     0     0    0      0       0      0       0   
4      0      0    0     0     0     0    0      0       0      0       0   

   אחרונים  אחרות  אחריה  אחריו  אחריות  אחריי  אחרים  אחרינו  אחרת  אחת  \
0        0      0      0      0       0      0      0       0     0    0   
1        0      1      0      0       0      0      0    

(76, 4188)

In [379]:
# SVC - Support Vector Classifier
svclassifier = SVC(kernel='linear')
svclassifier.fit(filtered_df, y_train)

SVC(kernel='linear')

In [336]:
print("final shape:", filtered_df.shape)
print("y_train shape:", y_train.shape)


final shape: (677, 4188)
y_train shape: (677,)


In [380]:
# Check the model
# from sklearn.metrics import classification_report
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

0.6052631578947368
              precision    recall  f1-score   support

           0       0.78      0.67      0.72        58
           1       0.27      0.39      0.32        18

    accuracy                           0.61        76
   macro avg       0.52      0.53      0.52        76
weighted avg       0.66      0.61      0.63        76



In [381]:
filtered_df.shape

(677, 4188)

### Save output to csv (optional)
After you're done save your output to the 'classification_results.csv' csv file.<br/>
We assume that the dataframe with your results contain the following columns:
* column 1 (left column): 'test_example_id'  - the same id associated to each of the test stories to be predicted.
* column 2 (right column): 'predicted_category' - the predicted gender value for each of the associated story. 

Assuming your predicted values are in the `df_predicted` dataframe, you should save you're results as following:

In [382]:
# Classify test examples
df3 = filtered_df;
df3 = df3[0:323]
for col in df3.columns:
    df3[col].values[:] = 0

for index, row in df3.iterrows():
    hebrew_text = df_test['story'].iloc[index]

    tokens = ht.tokenize(hebrew_text) 
    for grp, token, token_num, (start_index, end_index) in tokens:
        if token in df3:
            df3[token].iloc[index] = df3[token].iloc[index] + 1
        
df3.head()
df3.shape

אבא  אבי  אביב  אבן  אבק  אגב  אגם  אדום  אדומות  אדם  אדמה  אה  אהב  אהבה  \
0    0    0     0    0    0    0    0     0       0    0     0   0    0     0   
1    0    0     0    0    0    0    0     0       0    0     0   0    0     0   
2    0    0     0    0    0    0    0     0       0    0     0   0    0     0   
3    0    0     0    0    0    0    0     0       0    0     0   0    0     0   
4    0    0     0    0    0    0    0     0       0    0     0   0    0     0   

   אהבו  אהבתי  אהיה  או  אוגוסט  אודות  אוהב  אוהבים  אוהבת  אוהד  אוהדים  \
0     0      0     0   0       0      0     0       0      0     0       0   
1     0      0     0   0       0      0     0       0      0     0       0   
2     0      0     0   0       0      0     0       0      0     0       0   
3     0      0     0   0       0      0     0       0      0     0       0   
4     0      0     0   0       0      0     0       0      0     0       0   

   אוהל  אוהלים  אוויר  אווירה  אוטו  אוטובוס  אוטומטי  אוכל  אוכלים  אולי  \
0     0       0      0       0     0        0        0     0       0     0   
1     0       0      0       0     0        0        0     0       0     0   
2     0       0      0       0     0        0        0     0       0     0   
3     0       0      0       0     0        0        0     0       0     0   
4     0       0      0       0     0        0        0     0       0     0   

   אולם  אולמות  אומנה  אומנם  אומץ  אומר  אומרים  אומרת  אונים  אונליין  \
0     0       0      0      0     0     0       0      0      0        0   
1     0       0      0      0     0     0       0      0      0        0   
2     0       0      0      0     0     0       0      0      0        0   
3     0       0      0      0     0     0       0      0      0        0   
4     0       0      0      0     0     0       0      0      0        0   

   אופטימיים  אופי  אופייני  אופן  אופנוע  אופניים  אופציה  אוציא  אוקיי  אור  \
0          0     0        0     0       0        0       0      0      0    1   
1          0     0        0     0       0        0       0      0      0    0   
2          0     0        0     0       0        0       0      0      0    0   
3          0     0        0     0       0        0       0      0      0    0   
4          0     0        0     0       0        0       0      0      0    0   

   אורז  אורח  אורחים  אורך  אושר  אות  אותה  אותו  אותך  אותם  אותן  אותנו  \
0     0     0       0     0     0    0     0     0     0     0     0      0   
1     0     0       0     0     0    0     0     0     0     1     0      0   
2     0     0       0     0     0    0     2     0     0     1     0      2   
3     0     0       0     0     0    0     0     0     0     1     0      5   
4     0     0       0     0     0    0     0     1     0     0     0      2   

   אז  אזור  אזורים  אזכה  אזכור  אח  אחווה  אחוז  אחוזים  אחורה  אחות  אחותה  \
0   1     0       0     0      0   0      0     0       0      0     0      0   
1   0     0       0     0      0   0      0     2       0      0     0      0   
2   0     0       0     0      0   0      0     0       0      0     0      0   
3   0     0       0     0      0   0      0     0       0      0     0      0   
4   2     0       0     0      0   0      0     0       0      0     0      0   

   אחותי  אחזור  אחי  אחיי  אחים  אחלה  אחר  אחראי  אחראית  אחרון  אחרונה  \
0      0      0    0     0     0     0    0      0       0      0       0   
1      0      0    0     0     0     0    0      0       1      0       0   
2      0      0    0     0     1     0    0      0       0      0       0   
3      0      0    0     0     0     1    0      0       0      0       0   
4      0      0    0     0     0     0    0      0       0      0       0   

   אחרונים  אחרות  אחריה  אחריו  אחריות  אחריי  אחרים  אחרינו  אחרת  אחת  \
0        0      0      0      0       0      0      0       0     0    1   
1        0      0      0      0       0      0      0    

(323, 4188)

In [383]:
#Additional Machine learning model - Svc
svclassifier = SVC(kernel='linear')
svclassifier.fit(filtered_df, y_train)

SVC(kernel='linear')

In [384]:
selected_columns = df_test["test_example_id"]
id_copy = selected_columns.copy()
id_col = pd.DataFrame(id_copy,columns=['test_example_id'])

y_pred = svclassifier.predict(df3)
category = pd.DataFrame(y_pred,columns=['predicted_category'])


Total = category['predicted_category'].sum()
print (Total)

category['predicted_category'] = category['predicted_category'].replace({0:'m', 1:'f'})

category.head()
category.shape

id_col.head()
id_col.shape

df_predicted = pd.concat([id_col.reset_index(drop=True),category.reset_index(drop=True)], axis=1)

df_predicted.head()
df_predicted.shape

92


,predicted_category
0,m
1,m
2,f
3,m
4,f


(323, 1)

,test_example_id
0,0
1,1
2,2
3,3
4,4


(323, 1)

,test_example_id,predicted_category
0,0,m
1,1,m
2,2,f
3,3,m
4,4,f


(323, 2)

In [385]:
df_predicted.to_csv('classification_results.csv',index=False)